**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri May  1 07:12:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
'AJAX with Django 3.x.gslides'	 model-ICNet-trained-model.h5
 certificate.JPG		'NANDINI BANSAL.pdf'
'Colab Notebooks'		 Nandini_Resume_2020.pdf
 Companies_ML_DL_DS.gsheet	 siamese-text-similarity.zip
 dev.csv			 train.csv
'Getting started.pdf'		'Untitled Diagram (1).drawio'
 glove.6B.50d.txt		'Untitled Diagram.drawio'
'HUSE Implementation'		 yolov3


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13221, done.
remote: Total 13221 (delta 0), reused 0 (delta 0), pack-reused 13221
Receiving objects: 100% (13221/13221), 11.91 MiB | 12.12 MiB/s, done.
Resolving deltas: 100% (9041/9041), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [0]:
!echo "License" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-05-01 07:17:04--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  2.18MB/s    in 1m 49s  

2020-05-01 07:18:54 (1.42 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/20.jpg         
  inflating: data/obj/20.txt         
 extracting: data/obj/22403415607-7d842000e2-b.jpg  
 extracting: data/obj/22403415607-7d842000e2-b.txt  
  inflating: data/obj/7.jpg          
  inflating: data/obj/7.txt          
  inflating: data/obj/75ad7d9a68b8dbac61cf0210fb5159b6.jpg  
 extracting: data/obj/75ad7d9a68b8dbac61cf0210fb5159b6.txt  
  inflating: data/obj/7dab210d4d37b58c34e17347d57d4d36.jpg  
 extracting: data/obj/7dab210d4d37b58c34e17347d57d4d36.txt  
  inflating: data/obj/car-embossed.jpg  
  inflating: data/obj/car-embossed.txt  
  inflating: data/obj/Car-Image-with-Indian-Number-Plate.png  
 extracting: data/obj/Car-Image-with-Indian-Number-Plate.txt  
  inflating: data/obj/Car-Number-Plate.jpg  
  inflating: data/obj/Car-Number-Plate.txt  
  inflating: data/obj/classes.txt    
  inflating: data/obj/d2.jpg         
  inflating: data/obj/d2.txt         
  inflating: data/obj/d4.jpg         
  inflating

In [11]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.483398 0.697810 0.197266 0.067153

['15', '0.483398', '0.697810', '0.197266', '0.067153']
0 data/obj/images_180.txt
0 0.483398 0.697810 0.197266 0.067153
15 0.461000 0.587302 0.354000 0.132275

['15', '0.461000', '0.587302', '0.354000', '0.132275']
1 data/obj/images_77.txt
0 0.461000 0.587302 0.354000 0.132275
15 0.507394 0.695035 0.238447 0.075650

['15', '0.507394', '0.695035', '0.238447', '0.075650']
2 data/obj/images_186.txt
0 0.507394 0.695035 0.238447 0.075650
15 0.796000 0.638408 0.140000 0.065744

['15', '0.796000', '0.638408', '0.140000', '0.065744']
3 data/obj/images_207.txt
0 0.796000 0.638408 0.140000 0.065744
15 0.660656 0.392012 0.655738 0.298817

['15', '0.660656', '0.392012', '0.655738', '0.298817']
4 data/obj/images_128.txt
0 0.660656 0.392012 0.655738 0.298817
15 0.412879 0.490909 0.125758 0.059091

['15', '0.412879', '0.490909', '0.125758', '0.059091']
5 data/obj/images_107.txt
0 0.412879 0.490909 0.125758 0.059091
15 0.490000 0.606127 0.456000 0.140044

['15', 

In [14]:
import glob
images_list = glob.glob("data/obj/*.jpg")
images_list2 = glob.glob("data/obj/*.png")
images_list.extend(images_list2)
print(images_list)
print(len(images_list))

['data/obj/DbH5xTRXkAAsLoy.jpg', 'data/obj/images_223.jpg', 'data/obj/images_93.jpg', 'data/obj/images_218.jpg', 'data/obj/images_16.jpg', 'data/obj/images_210.jpg', 'data/obj/images_237.jpg', 'data/obj/images_183.jpg', 'data/obj/Skoda-Kodiaq-India-Review-15.jpg', 'data/obj/images_155.jpg', 'data/obj/images_182.jpg', 'data/obj/images_14.jpg', 'data/obj/images_82.jpg', 'data/obj/images_204.jpg', 'data/obj/india-uttarakhand-rishikesh-car-plate-close-up-BK64DW.jpg', 'data/obj/images_162.jpg', 'data/obj/images_230.jpg', 'data/obj/images_205.jpg', 'data/obj/main-cropped.jpg', 'data/obj/images_31.jpg', 'data/obj/images_211.jpg', 'data/obj/images_132.jpg', 'data/obj/images_163.jpg', 'data/obj/images_157.jpg', 'data/obj/images_7.jpg', 'data/obj/images_209.jpg', 'data/obj/images_141.jpg', 'data/obj/images_207.jpg', 'data/obj/images_123.jpg', 'data/obj/images_169.jpg', 'data/obj/vehicle-with-a-licence-plate-from-the-indian-state-of-sikkim.jpg', 'data/obj/images_173.jpg', 'data/obj/images_106.jpg

In [0]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [0]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show